In [1]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 1: Khởi tạo

import pandas as pd
import os
import glob
import openmeteo_requests
import requests_cache
from retry_requests import retry
from datetime import datetime
import time

print("Cell 1: Môi trường sẵn sàng.")

Cell 1: Môi trường sẵn sàng.


In [2]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 2: Đọc Metadata

metadata_filename = "stations_metadata.csv"
if not os.path.exists(metadata_filename):
    print(f"LỖI: File '{metadata_filename}' không tồn tại.")
else:
    df_metadata = pd.read_csv(metadata_filename)
    print(f"Đã đọc thành công thông tin của {len(df_metadata)} trạm từ '{metadata_filename}'.")

Đã đọc thành công thông tin của 31 trạm từ 'stations_metadata.csv'.


In [3]:
# NOTEBOOK OPEN-METEO WEATHER
# CELL 3: Tải và Lưu file Weather Raw (TỪ 2022)

if 'df_metadata' not in locals():
    print("Vui lòng chạy Cell 2 trước.")
else:
    cache_session = requests_cache.CachedSession('.cache_weather', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    weather_output_dir = "hanoi_weathermeteo_from04Aug_data_raw"
    if not os.path.exists(weather_output_dir): os.makedirs(weather_output_dir)

    # --- KHOẢNG THỜI GIAN THỰC TẾ CÓ DỮ LIỆU LÀ TỪ NGÀY 04 THÁNG 8 NĂM 2022---
    START_DATE_FIXED = "2022-08-02"
    END_DATE_FIXED = "2025-10-19"
    # --------------------------------------------

    print(f"\nBắt đầu tải dữ liệu Khí tượng cho {len(df_metadata)} vị trí từ {START_DATE_FIXED} đến {END_DATE_FIXED}...")

    for index, station in df_metadata.iterrows():
        loc_id = station['location_id']
        lat = station['lat']
        lon = station['lon']
        
        output_filename = os.path.join(weather_output_dir, f"weather_meteo_{loc_id}.csv")
        if os.path.exists(output_filename):
            print(f"\n--- File cho trạm ID: {loc_id} đã tồn tại. Bỏ qua. ---")
            continue

        print(f"\n--- Đang lấy dữ liệu Khí tượng cho vị trí của trạm ID: {loc_id} ---")

        # Sử dụng API Lịch sử ("/archive")
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat, "longitude": lon,
            "start_date": START_DATE_FIXED, "end_date": END_DATE_FIXED,
            "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "wind_speed_10m", "wind_direction_10m", "pressure_msl", "boundary_layer_height"],
            "timezone": "Asia/Bangkok", # ⚠️ NOTE QUAN TRỌNG: timezone phải là "Asia/Bangkok" để đồng bộ UTC+7 (Việt Nam)
        }
        
        try:
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]
            
            hourly = response.Hourly()
            hourly_data = {
                "datetime": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True), # Thêm utc=True
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True), # Thêm utc=True
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                    )
                }
            
            for i, var_name in enumerate(params["hourly"]):
                values = hourly.Variables(i).ValuesAsNumpy()
                hourly_data[var_name] = values[:len(hourly_data["datetime"])]

            df_weather = pd.DataFrame(data=hourly_data)
            # Chuyển đổi từ UTC sang múi giờ Việt Nam
            df_weather["datetime"] = df_weather["datetime"].dt.tz_convert("Asia/Bangkok")
            df_weather.to_csv(output_filename, index=False, encoding='utf-8-sig')
            
            print(f"  -> Đã lưu thành công {len(df_weather)} giờ dữ liệu vào file {output_filename}")
        except Exception as e:
            print(f"  -> Đã xảy ra lỗi: {e}")
        
        finally:
            # --- LOGIC MỚI: NGHỈ SAU MỖI LẦN GỌI ---
            # Thêm một khoảng nghỉ 8 giây sau mỗi lần gọi API, dù thành công hay thất bại
            # để đảm bảo không vượt quá giới hạn.
            print("    -> Nghỉ 8 giây...")
            time.sleep(8)
            # ----------------------------------------
            
    print("\n\n---> HOÀN TẤT TẢI DỮ LIỆU KHÍ TƯỢNG <---")


Bắt đầu tải dữ liệu Khí tượng cho 31 vị trí từ 2022-08-02 đến 2025-10-19...

--- Đang lấy dữ liệu Khí tượng cho vị trí của trạm ID: 2539 ---
  -> Đã lưu thành công 28200 giờ dữ liệu vào file hanoi_weathermeteo_from04Aug_data_raw/weather_meteo_2539.csv
    -> Nghỉ 8 giây...

--- Đang lấy dữ liệu Khí tượng cho vị trí của trạm ID: 7441 ---
  -> Đã lưu thành công 28200 giờ dữ liệu vào file hanoi_weathermeteo_from04Aug_data_raw/weather_meteo_7441.csv
    -> Nghỉ 8 giây...

--- Đang lấy dữ liệu Khí tượng cho vị trí của trạm ID: 1285357 ---
  -> Đã lưu thành công 28200 giờ dữ liệu vào file hanoi_weathermeteo_from04Aug_data_raw/weather_meteo_1285357.csv
    -> Nghỉ 8 giây...

--- Đang lấy dữ liệu Khí tượng cho vị trí của trạm ID: 2161290 ---
  -> Đã lưu thành công 28200 giờ dữ liệu vào file hanoi_weathermeteo_from04Aug_data_raw/weather_meteo_2161290.csv
    -> Nghỉ 8 giây...

--- Đang lấy dữ liệu Khí tượng cho vị trí của trạm ID: 2161291 ---
  -> Đã lưu thành công 28200 giờ dữ liệu vào file h

In [4]:
# NOTEBOOK OPEN-METEO WEATHER
# CELL 4: Gộp các file Weather Raw

weather_output_dir = "hanoi_weathermeteo_from04Aug_data_raw"

if not os.path.exists(weather_output_dir):
    print(f"Thư mục '{weather_output_dir}' không tồn tại.")
else:
    all_csv_files = glob.glob(os.path.join(weather_output_dir, "*.csv"))
    if not all_csv_files:
        print("Không tìm thấy file CSV nào để gộp.")
    else:
        print(f"Tìm thấy {len(all_csv_files)} file Weather CSV. Bắt đầu gộp...")
        
        all_data_frames = []
        for f in all_csv_files:
            df = pd.read_csv(f)
            loc_id = int(os.path.basename(f).replace('weather_meteo_', '').replace('.csv', ''))
            df['location_id'] = loc_id
            all_data_frames.append(df)
            
        final_df = pd.concat(all_data_frames, ignore_index=True)
        final_df['datetime'] = pd.to_datetime(final_df['datetime'])
        final_df = final_df.sort_values(by=['location_id', 'datetime'])
        
        final_output_filename = "hanoi_weathermeteo_from04Aug_COMBINED.csv"
        final_df.to_csv(final_output_filename, index=False, encoding='utf-8-sig')
        
        print("\n---> QUÁ TRÌNH GỘP HOÀN TẤT <---")
        print(f"Dữ liệu Weather đã được gộp và lưu vào file '{final_output_filename}'")

Tìm thấy 31 file Weather CSV. Bắt đầu gộp...

---> QUÁ TRÌNH GỘP HOÀN TẤT <---
Dữ liệu Weather đã được gộp và lưu vào file 'hanoi_weathermeteo_from04Aug_COMBINED.csv'
